# Calculate effective reproductive number $R_e$

In [1]:
import pandas as pd
import numpy as np

In [2]:
IMPORT_HUMANS_PATH = '../outputs/humans_infected_time_course.csv'

In [3]:
humans = pd.read_csv(IMPORT_HUMANS_PATH, sep = ';')

In [4]:
humans.head()

,h_ID,loc,status,WasInfected,Diagnosed,Hospitalized,ICUed,time,place_of_infection,time_of_infection,got_infected_by
0,187,73,S,0,0,0,0,1,NaN,NaN,NaN
1,1087,389,S,0,0,0,0,1,NaN,NaN,NaN
2,194,75,S,0,0,0,0,1,NaN,NaN,NaN
3,1088,389,S,0,0,0,0,1,NaN,NaN,NaN
4,188,74,S,0,0,0,0,1,NaN,NaN,NaN


In [5]:
humans.shape

(160000, 11)

We can only thoroughly calculate $R_e$ from closed cases (`R` and `D`).

In [6]:
closed_IDs = np.unique(humans[humans.status.isin(['R', 'D'])].h_ID.values)

In [7]:
closed_ID = closed_IDs[0]
closed_ID

1

In [8]:
humans[humans.got_infected_by == closed_ID]['got_infected_by'].count()

2

So this human infected two others.

In [9]:
humans[humans.got_infected_by.isin(closed_IDs)].groupby('got_infected_by').count().h_ID

got_infected_by
1.0       2
2.0       1
7.0       1
10.0      3
15.0      1
         ..
1591.0    1
1593.0    2
1597.0    1
1598.0    1
1600.0    1
Name: h_ID, Length: 767, dtype: int64

In [10]:
humans[humans.got_infected_by.isin(closed_IDs)].groupby('got_infected_by').count().h_ID.describe()

count    767.000000
mean       2.063885
std        1.649381
min        1.000000
25%        1.000000
50%        1.000000
75%        2.500000
max       13.000000
Name: h_ID, dtype: float64

$R_e$:

In [11]:
humans[humans.got_infected_by.isin(closed_IDs)].groupby('got_infected_by').count().h_ID.mean()

2.0638852672750976

Wrap all of this into a function:

In [12]:
def effective_reproduction_number(humans):
    closed_IDs = np.unique(humans[humans.status.isin(['R', 'D'])].h_ID.values)
    r_e = humans[humans.got_infected_by.isin(closed_IDs)].groupby('got_infected_by').count().h_ID.mean()
    
    return r_e

In [13]:
effective_reproduction_number(humans)

2.0638852672750976

Condition on time; only allow infections that happened at or after `time`:

In [14]:
def effective_reproduction_number_conditional(humans, time):
    closed_IDs = np.unique(humans[humans.status.isin(['R', 'D'])].h_ID.values)
    
    relevant_humans = humans[humans.time_of_infection >= time]
    r_e = relevant_humans[relevant_humans.got_infected_by.isin(closed_IDs)].groupby('got_infected_by').count().h_ID.mean()
    
    return r_e

In [15]:
effective_reproduction_number_conditional(humans, 20)

1.1682242990654206